In [1]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromMeshVertices
from Eval.Clouds.Sphere import CreateBaseSpherePointCloud
from Eval.Error import CalculateRMSE

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ImportError: cannot import name 'CalculateRMSE2' from 'Eval.Error' (d:\programming\projects\dissertation-2022_2023\dissertation\Eval\Error.py)

In [ ]:
# Create Base Clouds for Comparison, Created Programatically for High Accuracy.

baseSphere = CreateBaseSpherePointCloud(diameter = 1.0, resolution=5000)
baseSphere.paint_uniform_color([0.5, 0.5, 0.5])


In [ ]:
# Get the Obtained Data Clouds for Comparison, on the Base Programatically Created Clouds.

evalDataDir = "../data/point_e/P-E_Sphere.ply"

targetSphere = CreatePointcloudFromMeshVertices(evalDataDir)
targetSphere.paint_uniform_color([1, 0, 0])

In [ ]:
# Quick Visualization of both the Obtained Cloud and the Base Cloud.

o3d.visualization.draw_geometries_with_editing([baseSphere, targetSphere], width=800, height=600)

In [ ]:
# Calculate the RMSE between the two clouds.

testSphere = CreateBaseSpherePointCloud(diameter = 1.25, resolution=3000)

rmse = CalculateRMSE2(baseSphere, testSphere)

print(f'RMSE: {rmse}')

In [ ]:
# Calculate the RMSE between the two clouds, using Open3D Registration.

# This is the same as the above, but using Open3D's Registration Functionality.
# This is a more accurate way of calculating the RMSE, as it takes into account
# the rotation and translation of the two clouds.

icpRegResults = o3d.pipelines.registration.registration_icp(baseSphere, targetSphere, 0.1, np.identity(4), o3d.pipelines.registration.TransformationEstimationPointToPoint(), o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))

print(icpRegResults) # Provides Inter_RMSE